<div style="width:100%; overflow:hidden; background-color:#F1F1E6; padding: 10px; border-style: outset; color:#17469e">
    <div style="width: 80%; float: left;">
    <h2 align="center">Universidad de Sonora</h2>
    <hr style="border-width: 3px; border-color:#17469e">
          <h1>Reconocimiento de patrones: Preparación de los datos</h1>          
          <h4>Ramón Soto C. <a href="mailto:rsotoc@moviquest.com/">(rsotoc@moviquest.com)</a></h4>
    </div>
    <div style="float: right;">
    <img src="images/escudo_unison.png">
    </div>
</div>

## Caso de estudio: [*Stack Overflow 2018 Developer Survey*](https://www.kaggle.com/stackoverflow/stack-overflow-2018-developer-survey)

Como caso de estudio principal en el presente curso hemos seleccionado la encuesta de desarrolladores 2018 de *Stack Overflow* disponible en [Kaggle](https://www.kaggle.com). En este esta etapa realizaremos el análisis de agrupamientos.

### 4. Modelado - ISODATA

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
... ahora utilizamos la técnica ISODATA para identificar prototipos de clases. <br>Inicializamos el contexto y cargamos los datos:
</div>

In [ ]:
"""
Reconocimiento de patrones: ISODATA
"""

#from scipy.spatial.distance import squareform

# Inicializar el ambiente
import sys
import numpy as np
import pandas as pd
import json
import pickle
#import math
import random
#import time

from IPython.display import display, HTML
from collections import Counter
from operator import itemgetter
#from scipy.spatial.distance import euclidean, pdist, squareform

np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1
pd.set_option('display.max_columns', 130)
pd.set_option('max_colwidth', 80)

LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales

In [ ]:
path = "Data sets/Stack Overflow Survey/"

# Recuperar encabezados de columnas en orden original
with open(path + 'survey_results_public_transformed.headers', 'rb') as file:  
    headers = pickle.load(file)

# Recuperar diccionarios... sólo por si se requieren
with open(path + 'survey_results_public_transformed.dicts', 'rb') as file:  
    dict_of_dicts = pickle.load(file)

with open(path + 'survey_results_public_transformed.json') as f:
    dict_json = json.load(f)
df = pd.DataFrame.from_dict(dict_json)
#df = df.sample(n=1000).reset_index(drop=True)

# Reordenar las columnas de acuerdo al orden original
df = df.reindex(headers, axis=1)

DATA_LEN = df.shape[0]

# Agregar una columna "cluster" inicializada a null 
df["Cluster"] = np.nan

In [ ]:
var_str = ['Hobby', 'OpenSource', 'Country', 'Student', 'Employment', 'FormalEducation', 
         'UndergradMajor', 'CompanySize', 'YearsCoding', 'YearsCodingProf', 'UpdateCV', 
         'JobSatisfaction', 'CareerSatisfaction', 'HopeFiveYears', 'JobSearchStatus', 
         'LastNewJob', 'TimeFullyProductive', 'AgreeDisagree1', 'AgreeDisagree2', 
         'AgreeDisagree3', 'OperatingSystem', 'NumberMonitors', 'CheckInCode', 'AdBlocker', 
         'AdBlockerDisable', 'AdsAgreeDisagree1', 'AdsAgreeDisagree2', 'AdsAgreeDisagree3', 
         'AIDangerous', 'AIInteresting', 'AIResponsible', 'AIFuture', 'EthicsChoice', 
         'EthicsReport', 'EthicsResponsible', 'EthicalImplications', 'HoursComputer', 
         'StackOverflowRecommend', 'StackOverflowVisit', 'StackOverflowHasAccount', 
         'StackOverflowParticipate', 'StackOverflowJobs', 'StackOverflowDevStory', 
         'StackOverflowJobsRecommend', 'StackOverflowConsiderMember', 'HypotheticalTools1', 
         'HypotheticalTools2', 'HypotheticalTools3', 'HypotheticalTools4', 'WakeTime', 
         'HypotheticalTools5', 'HoursOutside', 'SkipMeals', 'Exercise', 'EducationParents', 
         'Age', 'Dependents', 'SurveyTooLong', 'SurveyEasy']
var_list = ['DevType', 'CommunicationTools', 'EducationTypes', 'SelfTaughtTypes', 
         'HackathonReasons', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 
         'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 
         'FrameworkWorkedWith', 'FrameworkDesireNextYear', 'IDE', 'Methodology', 
         'VersionControl', 'AdBlockerReasons', 'AdsActions', 'ErgonomicDevices', 
         'RaceEthnicity', 'LanguageWorkedWith']
var_ranks = ['AssessJob', 'AssessBenefits', 'JobContactPriorities', 'JobEmailPriorities', 
             'AdsPriorities']
var_float = 'ConvertedSalary'

def distance_qual(x, y):
    # Número de variables; si var_float es array, modificar "+ 1" por "+ len(var_float)"
    numvars = len(var_str) + len(var_list) + len(var_ranks) + 1
    
    distancia = abs(x.ConvertedSalary - y.ConvertedSalary)
    if pd.isnull(distancia):
        distancia = 0
        numvars -= 1
        
    for col in var_str:
        if x[col] != y[col]:
            distancia += 1
        
    for col in var_list:
        num_vars = len(x[col]) + len(y[col])
        d = 0
        if num_vars > 0:
            d = (2*len(set(x[col] + y[col])) - num_vars) / num_vars
        distancia += d

    for col in var_ranks:
        d = 0
        max_vars = max(len(x[col]), len(y[col]))
        if len(x[col]) != 0 and len(y[col]) != 0:
            for v in range(len(x[col])):
                if x[col][v] != y[col][v]:
                    d += 1
        else:
            d += max_vars
        
        if d != 0:
            d /= max_vars
        distancia += d

    return distancia / numvars
    
def decode(dataframe):
    new_df = dataframe.copy(deep=True)
    
    for col in var_str:
        if col in list(dataframe) and col in dict_of_dicts:
            for index, row in dataframe.iterrows():
                value = dict_of_dicts[col][row[col]]
                new_df.at[clusters.index[index], col] = value
                
    for index, row in dataframe.iterrows():
        new_df.at[clusters.index[index], 'ConvertedSalary'] = row['ConvertedSalary'] * 200000
    
    for col in var_list:
        if col in list(dataframe):
            for index, row in dataframe.iterrows():
                values_list = row[col].copy()
                for i in range(len(values_list)):
                    values_list[i] = dict_of_dicts[col][values_list[i]]
                new_df.at[clusters.index[index], col] = values_list
                
    return new_df

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
A continuación ejecutamos el algoritmo ISODATA:
</div>

1) Definir los valores de $k_{init}, n_{min}, I_{max}, \sigma_{max}, L_{min}$ y $P_{max}$:

In [ ]:
K_INIT = 7
N_MIN = 100
I_MAX = 10
S_MAX = .8 # La desviación estándar está normalizada
D_MAX = 3 # El cluster sólo se divide cuando hay al menos estas variables con s>S_MAX

L_MIN = .25 # Las distancia están normalizadas
P_MAX = 2

NUM_CLUSTERS = K_INIT # valor de k
iteration = 0

2) Seleccionar de manera arbitraria *k* puntos en el espacio de características como centros iniciales de los clusters (centroides o centros de masa).

In [ ]:
# Inicializar los centroides
centroids = df.sample(n=NUM_CLUSTERS).reset_index(drop=True)
display(centroids)

3) Asignar cada punto del conjunto de datos al cluster donde la distancia del punto al centroide es menor.

In [ ]:
def update_clusters():
    global NUM_CLUSTERS, centroids
    changed = False
    cluster_col_index = df.shape[1] - 1
    
    if TALK :
        print("Actualizando clusters")
    for index, row in df.iterrows():
        dists = []
        for i, r in centroids.iterrows():
            dists.append(distance_qual(row, r))
        cluster = np.argmin(dists)
        
        # Si hay cambio, realizarlo y levantar la bandera 'changed'
        if(pd.isnull(row['Cluster']) or row['Cluster'] != cluster):
            df.iloc[index, cluster_col_index] = cluster
            changed = True
            
    # Contabilizar los elementos en cada cluster   
    to_eliminate = []
    for i in range(NUM_CLUSTERS):
        members = df[df["Cluster"]==i].count()["Cluster"]
        if members < N_MIN:
            to_eliminate.append(i)
        if (TALK) : 
            print("El cluster ", i, " incluye ", members, "miembros.")
    if (TALK) : 
        print()

    if len(to_eliminate) > 0:
        if (TALK) : 
            print("Clusters a eliminar:", to_eliminate)
        
        # Eliminar los centroides seleccionados
        centroids.drop(to_eliminate, inplace=True)    
        centroids = centroids.reset_index(drop=True)
        
        # Reetiquetar los registros afectados
        eliminated = 0
        for i in to_eliminate:
            i_e = i - eliminated
            # Reetiquetar como Null los registros en cada cluster eliminado
            df.loc[df.Cluster == i_e, 'Cluster'] = np.nan
            # Recorrer las etiquetas para coincidir con los nuevos índices
            for cj in range(i_e + 1, NUM_CLUSTERS):
                df.loc[df.Cluster == cj, 'Cluster'] = cj - 1
            # Actualizar el número actual de centroides
            NUM_CLUSTERS -= 1
            eliminated += 1
            
#        if (TALK) : 
#            for i in range(NUM_CLUSTERS):
#                members = df[df["Cluster"]==i].count()["Cluster"]
#                print("El cluster ", i, " incluye ", members, "miembros.")

        changed = True
        
    if changed:
        if TALK : 
            faltantes = df[pd.isnull(df["Cluster"])].shape[0]
            if faltantes > 0:
                print("Faltan por clasificar", faltantes, "miembros.\n")
            else :
                print()
                
        # Reclasificar los registros afectados
        for index, row in df[pd.isnull(df["Cluster"])].iterrows():
            dists = []
            for i, r in centroids.iterrows():
                dists.append(distance_qual(row, r))
            df.iloc[index, cluster_col_index] = np.argmin(dists)
                
        # Contabilizar los elementos en cada cluster   
        if TALK : 
            for i in range(NUM_CLUSTERS):
                members = df[df["Cluster"]==i].count()["Cluster"]
                print("El cluster ", i, " incluye ", members, "miembros.")
            print()
        
    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()

4) Calcular los centroides a partir de los puntos en cada cluster. 

In [ ]:
def update_centroids():
    global centroids
    
    for cl_j in range(NUM_CLUSTERS):        
        # Seleccionar registros en el cluster cl_j
        df_clusterj = df[df["Cluster"] == cl_j]
        
        centroids.loc[centroids.index[cl_j]] = get_centroide(df_clusterj).loc[0]        
    return

def get_centroide(data):
    # Copiar estructura de la tabla
    df2 = pd.DataFrame(data=None, columns=data.columns)
    #df2.append(pd.Series([np.nan]), ignore_index = True)

    col = 'ConvertedSalary'
    df2.at[0, col] = data[col].mean()

    # Moda en las columnas 'simples' (en var_str)
    mode = data[var_str].mode()
    for col in mode:
        df2.at[0, col] = mode[col].values[0]

    # Moda en las columnas con listas de longitud variable (en var_list)
    for col in var_list:
        mean_len = 0
        vars_list = []
        for index, row in data.iterrows():
            mean_len += len(row[col])
            vars_list = vars_list + row[col]
        mean_len /= data.shape[0]
        counter = Counter(vars_list)
        mean_list = []
        for v in counter.most_common(round(mean_len + 0.5)):
            mean_list.append(v[0])
        df2.at[0, col] = mean_list


    # Moda en las columnas con listas de longitud fija (en var_ranks)
    ranges = [11, 12, 6, 8, 8]
    # Para cada variable en var_list, obtener el número de componentes en el vector
    # y el nombre de la columna
    for i, col in zip(range(len(ranges)), var_ranks):
        # Inicializar una matriz (lista de listas, en realidad), con tantos renglones como 
        # componentes tiene el vector de la variable. Cada renglón tiene todos los valores 
        # utilizados en cada posición del vector
        vars = []
        for j in range(ranges[i] - 1):
            vars.append([])

        # Recorrer todos los elementos actualmente en el cluster para rellenar la matriz
        for index, row in data.iterrows():
            # Si el vector de la variable no está vacío...
            if len(row[col]) > 0:
                # Para cada componente en el vector...
                for j in range(len(row[col])):
                    # Si no es 0
                    if row[col][j] != '0':
                        # Agregarla al renglón actual en la matriz
                        vars[j].append(row[col][j])

        
        # Contabilizar ocurrencias de cada componente. Crear una matriz con el orden para
        # cada componente como renglones
        most_commons = []
        for j in range(ranges[i] - 1):
            counter = Counter(vars[j])
            #most_commons.append(counter.most_common(ranges[i] - 1))
            most_commons.append(counter.most_common())

        # Inicializar vector. Se escoge el valor más popular en la primera componente
        if len(most_commons) > 0 and len(most_commons[0]) > 0:
            vars_list = [most_commons[0][0][0]]
            # Para cada componente a partir de la segunda...
            for j in range(1, ranges[i] - 1):
                # Buscar la componente más común...
                for c in most_commons[j]:
                    # Siempre y cuando no esté utilizada...
                    if c[0] not in vars_list[:j]:
                        # Agregarla al vector y...
                        vars_list.append(c[0])
                        # Dejar de buscar.
                        break

        if len(vars_list) < ranges[i] - 1:
            for i in set(range(1, ranges[i])):
                if str(i) not in vars_list:
                    vars_list.append(str(i))
        df2.at[0, col] = vars_list

    return df2

# --------------------------
# Actualizar los centroides
update_centroids()

In [ ]:
display(centroids)

In [ ]:
deltas = []
delta = 0
def update_deltas():
    global deltas, delta, centroids
    deltas = [0] * NUM_CLUSTERS
    N = 0
    for j, rc in centroids.iterrows():
        n = 0
        for i, row in df[df["Cluster"]==j].iterrows():
            deltas[j] += distance_qual(row, rc)
            n += 1
        delta += deltas[j]
        deltas[j] /= n
        N += n
    delta /= N
    
    if TALK : 
        print("Las distancias medias en cada cluster son:\n", deltas)   
        print("\nLa distancia media promedio es:", delta)   
        
    return

update_deltas()

In [ ]:
import math

def std_dev():
    # Inicializar vector de desviaciones estándar... los valores actuales son inserbibles
    std_vectors = centroids.copy()
    
    for c in range(NUM_CLUSTERS) :
        df_c = df[(df["Cluster"]==c)]
        
        # Para cada variable numérica...
        df_cj = df_c[pd.notnull(df_c['ConvertedSalary'])]

        s = math.sqrt(sum(abs(df_cj["ConvertedSalary"] - 
                              centroids.iloc[c]["ConvertedSalary"])) / (df_cj.shape[0] - 1))
        std_vectors.loc[c, "ConvertedSalary"] = s
        
        for col in var_str:
            diff = sum(df_cj[col] != centroids.iloc[c][col])
            s = math.sqrt(diff / (df_cj.shape[0] - 1))
            std_vectors.loc[c, col] = s
        
        for col in var_list:
            y = centroids.iloc[c][col]
            diff = 0
            for i, row in df_cj.iterrows():
                x = row[col]
                num_vars = len(x) + len(y)
                if num_vars > 0:
                    diff += (2*len(set(x + y)) - num_vars) / num_vars
            s = math.sqrt(diff / (df_cj.shape[0] - 1))
            std_vectors.loc[c, col] = s
        
        for col in var_ranks:
            y = centroids.iloc[c][col]
            for i, row in df_cj.iterrows():
                diff = 0
                x = row[col]
                max_vars = max(len(x), len(y))
                if len(x) != 0 and len(y) != 0:
                    for v in range(len(x)):
                        if x[v] != y[v]:
                            diff += 1
                else:
                    diff += max_vars

                if diff != 0:
                    diff /= max_vars
            s = math.sqrt(diff / (df_cj.shape[0] - 1))
            std_vectors.loc[c, col] = s
         
    return std_vectors

display(std_dev())

In [ ]:
def divide_clusters():
    global NUM_CLUSTERS, centroids

    if TALK :
        display(centroids)
    
    # Cálculo de desviaciones estandar
    sigma_vect = std_dev()   
    if TALK :
        display(sigma_vect)
    
    candidates = []
    for c, s_row in sigma_vect.iterrows():
        for col in s_row:
            if col > S_MAX :
                candidates.append(c)
                break # Ya encontramos un atributo con sigma elevada 

    if TALK :
        print("Posibles clusters a dividir:", candidates)
    
    divided = False
    to_eliminate = []
    for c in candidates:
        members = df[df["Cluster"]==c].count()["Cluster"]
        cond = NUM_CLUSTERS < K_INIT/2 or (deltas[c] > delta and members > 2 * N_MIN)
        if cond: 
            d = 0
            # Obtener dos puntos "suficientemente separados", no es el óptimo, 
            # pero son buenos candidatos a buen costo
            count = 0
            while d < deltas[c] and count < 5000:
                s1 = df[df["Cluster"]==c].sample(n=2)
                d = distance_qual(s1.iloc[0], s1.iloc[1])
                count += 1
            if count < 5000:
                to_eliminate.append(c)
                centroids = centroids.append(s1)
                NUM_CLUSTERS += 1
            
    if len(to_eliminate) > 0 :
        if TALK : 
            print("Clusters a eliminar:", to_eliminate)
            print("")
        centroids.drop(to_eliminate, inplace=True)
        centroids = centroids.reset_index(drop=True)
        update_clusters()
        update_centroids()
        if TALK : 
            display(centroids)
            print("")
            
    return 

#divide_clusters()    

In [ ]:
def mix_clusters():
    global centroids, NUM_CLUSTERS
    
    # Matriz triangular superior de distancias entre centroides
    dist_lists = []
    for i, rc_i in centroids.iterrows():
        dist_lists.append([])
        for j, rc_j in centroids.iterrows():
            if j <= i:
                dist_lists[i].append(LARGER_DISTANCE)
            else:
                dist_lists[i].append(distance_qual(rc_i, rc_j))
    dist_matrix = np.array(dist_lists)
    
    to_eliminate = []
    # to_eliminate contendrá la mitad de los clusters unidos...
    while (dist_matrix.min() < LARGER_DISTANCE and len(to_eliminate) < P_MAX/2) :
        dist_min = dist_matrix.min()
        idx = (dist_matrix==dist_min).argmax()
        z1 = idx // len(centroids)
        z2 = idx % len(centroids)
        
        if dist_min < L_MIN:
            if TALK:
                print("Unificando clusters {} y {}".format(z1, z2))
                for i in range(NUM_CLUSTERS):
                    members = df[df["Cluster"]==i].count()["Cluster"]
                    print("El cluster ", i, " incluye ", members, "miembros.")
                print()

            # Modificar z1 para contener el centroide entre z1 y z2
            centroids.iloc[z1] = get_centroide(centroids.iloc[[z1, z2]]).loc[0]
            # Marcar puntos en z1 y z2 para reclasificar
            df.loc[df.Cluster == z1, 'Cluster'] = np.nan
            df.loc[df.Cluster == z2, 'Cluster'] = np.nan
            
            # Marcar z2 para eliminación
            to_eliminate.append(z2)
        
        dist_matrix[z1][z2] = LARGER_DISTANCE
        
    if len(to_eliminate) > 0:
        centroids.drop(to_eliminate, inplace=True)
        centroids = centroids.reset_index(drop=True)
        
        # Reetiquetar los registros afectados
        eliminated = 0
        for i in to_eliminate:
            i_e = i - eliminated
            # Recorrer las etiquetas para coincidir con los nuevos índices
            for cj in range(i_e + 1, NUM_CLUSTERS):
                df.loc[df.Cluster == cj, 'Cluster'] = cj - 1
            # Actualizar el número actual de centroides
            NUM_CLUSTERS -= 1
            eliminated += 1
            
        cluster_col_index = df.shape[1] - 1
        for index, row in df[pd.isnull(df["Cluster"])].iterrows():
            dists = []
            for i, r in centroids.iterrows():
                dists.append(distance_qual(row, r))
            df.iloc[index, cluster_col_index] = np.argmin(dists)
        update_centroids()
            
        if (TALK) : 
            # Contabilizar los elementos en cada cluster   
            for i in range(NUM_CLUSTERS):
                members = df[df["Cluster"]==i].count()["Cluster"]
                print("El cluster ", i, " incluye ", members, "miembros")
            print()

    return

#mix_clusters()

In [ ]:
# Reproducido aquí para facilitar la ejecución
#iteration +=1 #usar si se está probando dividir/unir demostrativo

I_MAX_INT = 5 # Iteraciones permitidas en cada ciclo k-means

while iteration < I_MAX:
    if NUM_CLUSTERS <= K_INIT / 2 :
        update_deltas()
        divide_clusters()
    elif (iteration % 2 == 0 or NUM_CLUSTERS > 2 * K_INIT) :
        mix_clusters()
        
    step = 0
    while KEEP_WALKING and step < I_MAX_INT :
        KEEP_WALKING = update_clusters()
        update_centroids()
            
    iteration += 1
    
if TALK : 
    print ("No más cambios.")

In [ ]:
display(centroids)